In [34]:
import pandas as pd
from tqdm import tqdm
import numpy as np
#from konlpy.tag import mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib.parse import quote
import requests
import json
import matplotlib.pyplot as plt
%matplotlib inline
import re
import datetime
from datetime import timedelta

In [35]:
kospi = pd.read_csv("Project_ThematicInvest/Data/stock_data/kospi.csv", encoding='cp949')
kosdaq = pd.read_csv("Project_ThematicInvest/Data/stock_data/kosdaq.csv", encoding='cp949')

# Load Trained Model

In [36]:
theme_vectors_df = pd.read_csv('theme_vectors_xnorm.csv')

In [37]:
theme_vectors_df

,Unnamed: 0,themes,vectors
0,0,2차전지,"[-5728.885039504268, -1713.120836096663, 2425...."
1,1,3D프린터,"[-1601.962142577162, -5690.222019276283, 14515..."
2,2,4대강,"[-4492.6769370889815, 246.49643547526284, 3120..."
3,3,5G,"[-5324.962389420718, -2843.6464109625995, 8535..."
4,4,AR,"[4351.3103650522535, -178.3511861403931, 10230..."
...,...,...,...
262,262,홈쇼핑,"[2465.1961745460285, 2935.895441516419, 8783.9..."
263,263,화장품,"[-3903.077506332542, -3863.4404631393263, 1084..."
264,264,화학섬유,"[-321.29043307725806, -398.19858047622256, 247..."
265,265,흑사병,"[-11903.33893423264, -12734.967693026807, -313..."


In [38]:
theme_vectors_df['vectors'] = theme_vectors_df['vectors'].apply(lambda x: eval(x))

In [39]:
theme_vectors_df['vectors'] = theme_vectors_df['vectors'].apply(lambda x: np.array(x))

# Fit Today News

### Input

In [40]:
real_test = pd.read_csv("theme_contents_200_length_same.csv",index_col=0)
test_date = '2020-06-29'

In [41]:
ttok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)
#stop_words = ['으로', '로도', '지만', '에서', '려는', '하다', '은', '는', '이', '가', '을', '를', '수', '할']
v_dimension = 300
v_window = 8
model = Word2Vec.load('word2vec.model')

using cached model


In [42]:
# news vectors without normalization

def vectorize_without_normal(news):
    news_words = news.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    token = sp(news)
    final_tokens = token
    init_v = np.array([0.0]*v_dimension)
    for word in final_tokens:
        word_vectors = model.wv
        if word in word_vectors.vocab:
            v = model.wv[word]
            init_v = init_v + v
    return init_v

In [43]:
# test

def test(news):
    news_vec = vectorize_without_normal(news)
    result = []
    for theme in theme_vectors_df['vectors']:
        cosine = 1 - distance.cosine(theme, news_vec)
        result.append(cosine)
    new_df = pd.DataFrame(data=np.zeros([267,2]), columns=['Theme', 'Result'])
    new_df['Theme'] = theme_vectors_df['themes']
    new_df['Result'] = result

    new_df.sort_values('Result', ascending=False, inplace=True)
    return new_df
    

In [44]:
lst = []
lst2 = []
for i in tqdm(range(len(real_test))):
    a = real_test['contents'][i]
    result = test(a)
    result = result.sort_values(by='Result', ascending=False).reset_index(drop=True)
    theme = result['Theme'][0]
    cosine = result['Result'][0]
    lst.append(theme)
    lst2.append(cosine)
    

  1%|          | 402/50600 [00:05<11:17, 74.06it/s]/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 50600/50600 [10:12<00:00, 82.60it/s]


In [45]:
df = pd.DataFrame(list(zip(lst, lst2)), columns =['Theme', 'Similarity']) 
df = pd.concat([real_test,final_data],axis=1)

NameError: name 'final_data' is not defined

In [13]:
standard = 0.95
df_filtered = df[our_result['Similarity']>=standard].reset_index(drop=True)
df_filtered = df_filtered[['Theme','contents']]

NameError: name 'df' is not defined

In [ ]:
df_final = df_filtered.groupby('Theme').count().sort_values(by='contents',ascending=False)
df_final = df_final[(df_final['contents']>3) & (df_final['conetents']<10)]

# Make Portfolio

In [25]:
def target_company(themes):
    company_list=themes[themes['Theme']==them]['Company'].iloc[0]
    company_list=str(company_list).replace('[','').replace(']','').replace("\'",'').replace(' ', '').split(',')
    
    return company_list

In [34]:
def date_convert(date):
    end_date = date
    
    def date_by_deducting_business_days(from_date):
        business_days_to_deduct = 6
        current_date = from_date
        while business_days_to_deduct > 0:
            current_date += datetime.timedelta(days=-1)
            weekday = current_date.weekday()
            if weekday >= 5: # sunday = 6
                continue
            business_days_to_deduct -= 1
        return current_date

    convert_end_date = datetime.datetime.strptime(date,'%Y-%m-%d').date()
    start_date=datetime.date.strftime(date_by_deducting_business_days(convert_end_date),'%Y-%m-%d')
    
    return start_date, end_date

In [38]:
def date_format(d):
    d = str(d).replace('-', '.')
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    
    this_date = dt.date(yyyy, mm, dd)
    return this_date

In [39]:
def date_format(d=''):
    if d != '':
        this_date = pd.to_datetime(d).date()
    else:
        this_date = pd.Timestamp.today().date()
    return (this_date)

In [40]:
def stock_info(stock_cd):
    url_float = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + stock_cd
    source = urlopen(url_float).read()
    soup = bs(source, 'lxml')
    
    tmp = soup.find(id='cTB11').find_all('tr')[6].td.text
    tmp = tmp.replace('\r', '').replace('\n','').replace('\t','')
    tmp = re.split('/', tmp)
                   
    outstanding = tmp[0].replace(',','').replace('주','').replace(' ','')
    outstanding = int(outstanding)
                   
    floating = tmp[1].replace(' ','').replace('%','')
    floating = float(floating)
                   
    name = soup.find(id='pArea').find('div').find('div').find('tr').find('td').find('span').text
    
    stock_outstanding[stock_cd] = outstanding
    stock_floating[stock_cd] = floating
    stock_name[stock_cd] = name

In [44]:
def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    if start_date:
        start_date = date_format(start_date)
    else:
        start_date = dt.date.today()
    if end_date:
        end_date = date_format(end_date)+datetime.timedelta(days=-1)
    else:
        end_date = dt.date.today()
    
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_index).read()
    source = bs(source, 'lxml')
    
    dates = source.find_all('td', class_='date')
    prices = source.find_all('td', class_='number_1')
    
    for n in range(len(dates)):
        if dates[n].text.split('.')[0].isdigit():
            
            # Handling date
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:
                
                # Handling closing price
                this_close = prices[n*4].text
                this_close = this_close.replace(',','')
                this_close = float(this_close)
                
                historical_prices[this_date] = this_close

            elif this_date < start_date:
                return historical_prices
            
    
    # Navigating page
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        
        last_page = last_page.split('&')[1]
        last_page = last_page.split('=')[1]
        last_page = int(last_page)
        
    if page_n < last_page:
        page_n += 1
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)
        
    return historical_prices

In [45]:
def historical_stock_naver(stock_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    if start_date:
        start_date = date_format(start_date)
    else:
        start_date = dt.date.today()
    if end_date:
        end_date = date_format(end_date)+datetime.timedelta(days=-1)
    else:
        end_date = dt.date.today()
    
    naver_stock = 'http://finance.naver.com/item/sise_day.nhn?code=' + stock_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_stock).read()
    #source = requests.get(naver_stock)
    source = bs(source, 'lxml')
    
    dates = source.find_all('span', class_='tah p10 gray03')
    prices = source.find_all('td', class_='num')
    
    for n in range(len(dates)):
        
        if len(dates) > 0:
            
            # Handling date
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:
                
                # Handling closing price
                this_close = prices[n*6].text
                this_close = this_close.replace(',','')
                this_close = float(this_close)
                
                historical_prices[this_date] = this_close

            elif this_date < start_date:
                return historical_prices
            
    
    # Navigating page
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        last_page = last_page.split('&')[1]
        last_page = last_page.split('=')[1]
        last_page = int(last_page)
        
    if page_n < last_page:
        page_n += 1
        historical_stock_naver(stock_cd, start_date, end_date, page_n, last_page)
        
    return historical_prices

In [79]:
def threshold(data):
    if data < -0.05 or data > 0.05: return False
    else : return True

In [37]:
def target_theme(target_comapny):
    target_theme = {}
    for target in target_company:
        try:
            code = kosdaq[kosdaq['기업명']==target].iloc[0]['종목코드']
            code = str(code).zfill(6)
        except:
            code = kospi[kospi['기업명']==target].iloc[0]['종목코드']
            code = str(code).zfill(6)
        target_theme[code] = target
        
    return target_theme

027740 마니커
136480 하림
208140 정다운
072020 중앙백신
195500 마니커에프앤지
073560 우리손에프앤지
005940 NH투자증권
900340 윙입푸드
178780 유테크
015890 태경산업
003380 하림지주
198440 고려시멘트
027970 세하
060590 씨티씨바이오
027710 팜스토리
043200 파루
018620 우진비앤지
035290 더블유에프엠
041960 코미팜
033250 체시스
044960 이글벳
003540 대신증권
054670 대한뉴팜
086060 진바이오텍
036480 대성미생물
006980 우성사료
025870 신라에스지
001790 대한제당
353810 이지바이오
005930 삼성전자
052670 제일바이오
136490 선진


In [41]:
def stock_get_info(target_theme):
    stock_outstanding = dict()
    stock_floating = dict()
    stock_name = dict()
    for stock_cd in target_theme.keys():
        stock_info(stock_cd)
    
    return stock_outstanding, stock_floating, stock_name

027740
136480
208140
072020
195500
073560
005940
900340
178780
015890
003380
198440
027970
060590
027710
043200
018620
035290
041960
033250
044960
003540
054670
086060
036480
006980
025870
001790
353810
005930
052670
136490


In [46]:
def theme_historical_price(target_theme, stard_date, end_date):
    theme_historical_prices = dict()

    for stock_cd in target_theme.keys():
        historical_prices = dict()
        historical_stock_naver(stock_cd, start_date, end_date)

        theme_historical_prices[stock_cd] = historical_prices

    theme_historical_price = pd.DataFrame(theme_historical_prices)
    theme_historical_price.sort_index(axis=1, inplace=True)

    theme_historical_price = theme_historical_price.fillna(method='ffill')
    if theme_historical_price.isnull().values.any():
        theme_historical_price = theme_historical_price.fillna(method='bfill')

    theme_historical_price['kospi200'] = historical_index_naver('KPI200', start_date, end_date).values()
    
    return theme_historical_price

,001040,003240,023530,057050,138250
2020-06-29,84300.0,693000.0,76100.0,63800.0,12550.0
2020-06-26,85800.0,717000.0,78700.0,65400.0,13050.0
2020-06-25,85400.0,716000.0,77500.0,64500.0,13350.0
2020-06-24,88300.0,735000.0,79700.0,66900.0,13250.0
2020-06-23,87300.0,737000.0,79700.0,66000.0,12300.0
2020-06-22,88200.0,738000.0,81000.0,66100.0,12050.0


In [ ]:
def make_portfolio(theme, test_date):
    start_date, end_date = convert_date(test_date)
    themes=pd.read_csv("./Data/filtered_data/final_data.csv",index_col=0)
    target_company = target_company(themes)
    target_theme = target_theme(target_comapny)
    stock_outstanding, stock_floating, stock_name = stock_get_info(target_theme)
    theme_historical_price = theme_historical_price(target_theme, start_date, end_date)

    df = pd.DataFrame(np.zeros([6,len(target_company)+1]), columns=theme_historical_price.columns, index=theme_historical_price.index)
    for i in range(5):
        df.iloc[i] = (theme_historical_price.iloc[i] - theme_historical_price.iloc[i+1])/theme_historical_price.iloc[i]
    df.drop(index=[df.index[-1]], inplace=True)

    print('오늘의 추천종목은 : ')
    for i in range(len(target_company)):
        standard = 1
        for j in range(5):
            standard *= threshold(df.iloc[:, i][j])

        if standard == 1 : 
            print('{} : {}'.format(df.columns[i], target_theme[df.columns[i]]))

In [ ]:
for i in df_final.Theme:
    make_portfolio(i, test_date)